In [2]:
!git clone https://github.com/openai/shap-e.git

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 16.74 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [3]:
%cd shap-e

/content/shap-e


In [4]:
!pip install -e


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-e option requires 1 argument


In [5]:
pip install -e .

Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-spt__slh/clip_779363b5f6b34ed2834ccd723fc41945
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-spt__slh/clip_779363b5f6b34ed2834ccd723fc41945
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
 

In [6]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/content/shap-e/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/shap-e/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:12<00:00, 75.7MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

sample understanding of how it works

In [ ]:
batch_size = 4
guidance_scale = 15.0
prompt = "a shark"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIYAAFh1gE9wfUtvek5ue01ufFBteU5sdkpteUdse1BrdkdqeUZqfE…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAHCZnmKSl2mRkFSMkl2Kk1uJilSJk1GHkE6Fi1J+g1Z5fz9ygT…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAK2vrqyuramrq6epqKGjo6GjoqGioKGin6Cin6Cgnp6hnZ+fnp…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIYAAIOalGqNjGGCil15e1N6fVh3fVR2f1JyfUtxek5vek5veEpwdk…

In [ ]:
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


The prompt given and click on button and the obj will be generated.

In [9]:

#UI: prompt box + “Generate” button

import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink
from google.colab import files
from shap_e.util.notebooks import decode_latent_mesh

prompt_in = widgets.Text(
    value='a shark',
    placeholder='Type your text prompt here',
    description='Prompt:',
    layout=widgets.Layout(width='70%')
)
btn = widgets.Button(description='Generate 3D', button_style='primary')
output = widgets.Output()
display(prompt_in, btn, output)


# Callback to run your Shap‑E code, one batch only

def on_generate(_):
    with output:
        clear_output()
        prompt = prompt_in.value
        print(f"▶️ Generating for prompt: “{prompt}” …")

        # your existing settings
        batch_size = 4
        guidance_scale = 15.0
        render_mode = 'nerf'
        size = 64

        # 2.a) sample latents
        latents = sample_latents(
            batch_size=batch_size,
            model=model,
            diffusion=diffusion,
            guidance_scale=guidance_scale,
            model_kwargs=dict(texts=[prompt] * batch_size),
            progress=True,
            clip_denoised=True,
            use_fp16=True,
            use_karras=True,
            karras_steps=64,
            sigma_min=1e-3,
            sigma_max=160,
            s_churn=0,
        )

        # 2.b) render & display each as GIF
        cameras = create_pan_cameras(size, device)
        for i, latent in enumerate(latents):
            images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
            print(f"Result #{i}:")
            display(gif_widget(images))

        # 2.c) decode to meshes and save + link them
        print("\n💾 Saving meshes and providing download links:")
        for i, latent in enumerate(latents):
            mesh = decode_latent_mesh(xm, latent).tri_mesh()
            ply_path = f'example_mesh_{i}.ply'
            obj_path = f'example_mesh_{i}.obj'
            glb_path = f'example_mesh_{i}.glb'
            # open file handles and write
            with open(ply_path, 'wb') as f_ply:
                mesh.write_ply(f_ply)
            with open(obj_path, 'w') as f_obj:
                mesh.write_obj(f_obj)
            with open(glb_path, 'w') as glb_obj:
                mesh.write_obj(glb_obj)
            # Show direct Colab download links
            display(FileLink(ply_path))
            display(FileLink(obj_path))
            display(FileLink(glb_path))
        print("\n✅ Done!")

btn.on_click(on_generate)


Text(value='a shark', description='Prompt:', layout=Layout(width='70%'), placeholder='Type your text prompt he…

Button(button_style='primary', description='Generate 3D', style=ButtonStyle())

Output()